In [1]:

## BELL-LAPADULA POLICY MODEL ##

class CO: # класс объекта классификации
    class_order = ["unclassified", "classified", "secret", "top_secret"]
    def __init__(self, name, classification):
        self.name = name
        self.classification = classification
    def get_classification(self):
        return self.classification

class BLPUser(CO): ## пользователи
    def __init__(self, name, classification):
        CO.__init__(self, name, classification)

    def read(self, asset):
        if isinstance(asset, BLPAsset):
            return asset.get_message(self)

    def write(self, asset, new_message):
        if isinstance(asset, BLPAsset):
            return asset.set_message(self, new_message)

        
class BLPAsset(CO): ## доступы
    def __init__(self, name, message, classification):
        CO.__init__(self, name, classification)
        self.message = message

    def get_message(self, user): 
        if isinstance(user, BLPUser):
            if self.class_order.index((user.classification)) >= self.class_order.index(self.classification):
                return self.message
        return "Unable to read."
 
    def set_message(self, user, new_message):
        if isinstance(user, BLPUser):
            if self.class_order.index((user.classification)) <= self.class_order.index(self.classification):
                self.message = new_message
                return self.message
        return "Unable to write."

    


In [2]:
ts = BLPUser("Obj1", "top_secret")
s = BLPUser("Obj2", "secret")
c = BLPUser("Obj3", "classified")
u = BLPUser("Obj4", "unclassified")

tsa = BLPAsset("top_secret_asset", "This is a top secret message.", "top_secret")
sa = BLPAsset("secret_asset", "This is a secret message.", "secret")
ca = BLPAsset("classified asset", "This is a classified message.", "classified")
ua = BLPAsset("unclassified asset", "This is an unclassified message.", "unclassified")

print("Example 1: secret agent unable to read up")
print(s.read(tsa))
print(s.read(sa))
print(s.read(ca))
print(s.read(ua))

print("Example 2: top secret agent unable to write down")
print(ts.write(tsa, "I can write here."))
print(ts.write(sa, "I can write here."))
print(ts.write(ca, "I can write here."))
print(ts.write(ua, "I can write here."))

Example 1: secret agent unable to read up
Unable to read.
This is a secret message.
This is a classified message.
This is an unclassified message.
Example 2: top secret agent unable to write down
I can write here.
Unable to write.
Unable to write.
Unable to write.


Для создания системы уровней допусков, грифов секретности и матрицы доступа, мы можем воспользоваться принципом наименьших привилегий (Principle of Least Privilege). Этот принцип гласит, что каждый субъект должен иметь только те права доступа, которые необходимы для выполнения своих задач.

Сначала определим уровни доступа для каждого пользователя и грифы секретности для каждого объекта:

1. **Уровни доступа для пользователей (L[u]):**
   - \( L[u1] = l1 \) (Владелец или директор компании - уровень доступа top_secret)
   - \( L[u2] = l2 \) (администратор - уровень доступа secret)
   - \( L[u3] = l2 \) (сотрудник - уровень доступа classified)
   - \( L[u4] = l3 \) (непривилегированный пользователь - уровень доступа unclassified)

2. **Грифы секретности для объектов (L[o]):**
   - \( L[o1] = l2 \) (общесистемное ПО)
   - \( L[o2] = l1 \) (документ "Стратегические направления")
   - \( L[o3] = l2 \) (документ "График работы сотрудников")
   - \( L[o4] = l3 \) (специализированное ПО и БД )

Теперь создадим матрицу доступа \( A[u, o] \) с правами на чтение (r) и запись (w):


\begin{array}{cccc}
   & o1   & o2   & o3   & o4   \\
u1 & r, w & r, w & r, w & r, w \\
u2 & r, w & r    & r, w & r, w \\
u3 & r    & r    & r, w & r, w \\
u4 & r    &      &      & w    \\
\end{array}


Это означает, что каждый пользователь имеет права на чтение (r) и запись (w) ко всем объектам доступа в соответствии с их уровнем доступа и грифами секретности. Например, администратор системы (u1) имеет полные права (чтение и запись) ко всем объектам, включая общесистемное ПО (o1), документ "Стратегические направления развития организации" (o2) и так далее.